<a href="https://colab.research.google.com/github/irravasyuk/SQL_2/blob/master/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%BD%D0%B0_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель
* Використайте `optuna` для підбору найкращих параметрів



In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/Blog%20Ratings.csv")

df.head()

,blog_id,userId,ratings
0,9025,11,3.5
1,9320,11,5.0
2,9246,11,3.5
3,9431,11,5.0
4,875,11,2.0


In [2]:
df.describe()

,blog_id,userId,ratings
count,200140.000000,200140.000000,200140.000000
mean,5652.533621,2545.710158,3.117468
std,2970.685946,1446.195478,1.768113
min,1.000000,10.000000,0.500000
25%,2906.000000,1314.000000,2.000000
50%,5994.000000,2552.000000,3.500000
75%,8510.000000,3795.000000,5.000000
max,9755.000000,5010.000000,5.000000


In [3]:
!pip install -q surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
from surprise import Dataset, SVD, Reader

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[["blog_id", "userId", "ratings"]], reader)

In [11]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.8)

In [12]:
from surprise import BaselineOnly

bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

Estimating biases using als...


In [13]:
from surprise import SVD

alfo = SVD(n_factors=100,
           n_epochs=50,
           )

algo.fit(trainset)

Estimating biases using als...


In [14]:
from surprise import KNNBasic

algo = KNNBasic(k=20,
                min_k=1,
                sim_options={'name': 'cosine',
                             'user_based': True})

algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [15]:
from surprise import CoClustering

algo = CoClustering(n_cltr_u = 5,
                    n_cltr_i = 15,
                    n_epochs = 40)

algo.fit(trainset)

In [17]:
from surprise import BaselineOnly, accuracy


bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")

Estimating biases using als...
mae = 1.6355221813278011
mse = 3.4217291547247193
rmse= 1.8497916517069481


In [18]:
algo.predict(uid=2356,
             iid=5994)

Prediction(uid=2356, iid=5994, r_ui=None, est=3.4219534742219015, details={'was_impossible': False})

In [22]:
!pip install -q optuna

In [20]:
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split

def objective(trial):
    n_factors = trial.suggest_int('n_factors', 10, 200)
    n_epochs = trial.suggest_int('n_epochs', 5, 100)
    lr_all = trial.suggest_loguniform('lr_all', 1e-4, 1e-1)
    reg_all = trial.suggest_loguniform('reg_all', 1e-4, 1e-1)

    algo = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)

    trainset, testset = train_test_split(data, train_size=0.8)
    algo.fit(trainset)
    preds = algo.test(testset)
    rmse = accuracy.rmse(preds, verbose=False)
    return rmse

In [24]:
import optuna

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2024-09-03 12:17:47,059] A new study created in memory with name: no-name-dda07923-864c-4587-b831-81a2e0e380d6
<ipython-input-20-10f98fb38ff3>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr_all = trial.suggest_loguniform('lr_all', 1e-4, 1e-1)
<ipython-input-20-10f98fb38ff3>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  reg_all = trial.suggest_loguniform('reg_all', 1e-4, 1e-1)
[I 2024-09-03 12:17:54,576] Trial 0 finished with value: 2.0467492502237543 and parameters: {'n_factors': 19, 'n_epochs': 82, 'lr_all': 0.003155684868051925, 'reg_all': 0.0006299234132290761}. Best is trial 0 with value: 2.0467492502237543.
[I 2024-09-03 12:17:59,877] Trial 1 finished wi

In [25]:
best_params = study.best_trial.params
print('Best trial parameters:', best_params)

Best trial parameters: {'n_factors': 94, 'n_epochs': 5, 'lr_all': 0.00029774446411158235, 'reg_all': 0.09156957647115141}


In [26]:
best_algo = SVD(
    n_factors=best_params['n_factors'],
    n_epochs=best_params['n_epochs'],
    lr_all=best_params['lr_all'],
    reg_all=best_params['reg_all']
)

In [27]:
trainset, testset = train_test_split(data, train_size=0.8)
best_algo.fit(trainset)

In [28]:
preds = best_algo.test(testset)
print(f"Best model RMSE = {accuracy.rmse(preds, verbose=False)}")

Best model RMSE = 1.7759928679517105


In [29]:
preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 1.5225631972342795
mse = 2.961240011567706
rmse= 1.720825386716417
fcp = 0.5385053415123878
